<a href="https://colab.research.google.com/github/aakankshakadam97/Pharma_sales_analysis/blob/main/01Extract_mysql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to Mysql dump

In [1]:
#Upload and unzip file
from google.colab import files
import zipfile

# Upload the .zip file
uploaded = files.upload()


Saving sales.zip to sales.zip


In [2]:
# Unzip
zip_path = list(uploaded.keys())[0]  # get uploaded filename
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("extracted_sql")

!ls extracted_sql

sales.sql


In [3]:
#load .sql into sqlite
import sqlite3

In [5]:
# Install MySQL server
!apt-get install mysql-server > /dev/null
!service mysql start

# Set root password and create DB
!mysql -e "CREATE DATABASE mydb;" -u root

# Import dump (replace filename.sql with actual path)
!mysql -u root mydb < /content/extracted_sql/sales.sql

 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [6]:
#connect with python
!pip install mysql-connector-python

import mysql.connector
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 22.8 MB/s eta 0:00:00


In [10]:
!mysql -u root -e "DROP USER IF EXISTS 'colab'@'localhost';"
!mysql -u root -e "CREATE USER 'colab'@'localhost' IDENTIFIED WITH mysql_native_password BY 'colab123';"
!mysql -u root -e "GRANT ALL PRIVILEGES ON mydb.* TO 'colab'@'localhost';"
!mysql -u root -e "FLUSH PRIVILEGES;"


In [11]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
    host="localhost",
    user="colab",
    password="colab123",
    database="mydb"
)

df = pd.read_sql("SHOW TABLES;", conn)
df


/tmp/ipython-input-2401169417.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SHOW TABLES;", conn)


,Tables_in_mydb
0,det_sales
1,ms_product
2,ms_sales
3,transaction


In [12]:
#query the table and convert it into pandas dataframe
df = pd.read_sql("SELECT * FROM ms_product;", conn)
df.head()


/tmp/ipython-input-1770683612.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM ms_product;", conn)


,KD_OBAT,NAMA,SAT_JUAL,KD_PABRIK,HJ_RP
0,AI-0001,Vagizol Ovula 500mg,"45%,",suppos,0.0
1,AI-0002,4-Epeedo 10 mg/vial,vial,Kifa,184000.0
2,AI-0003,4-Epeedo 50 mg/vial,vial,Kifa,876750.0
3,AI-0004,Abingem 1000 mg,vial,Kifa,2040000.0
4,AI-0005,ABU Polivalen vial 5 ml,vial,Biof,382000.3


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6878 entries, 0 to 6877
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   KD_OBAT    6878 non-null   object 
 1   NAMA       6878 non-null   object 
 2   SAT_JUAL   6878 non-null   object 
 3   KD_PABRIK  6878 non-null   object 
 4   HJ_RP      6878 non-null   float64
dtypes: float64(1), object(4)
memory usage: 268.8+ KB


In [14]:
# Get list of all tables
tables = pd.read_sql("SHOW TABLES;", conn)
table_names = tables.iloc[:,0].tolist()

# Export each table to CSV
for table in table_names:
    df = pd.read_sql(f"SELECT * FROM {table};", conn)
    df.to_csv(f"/content/{table}.csv", index=False)
    print(f"Exported {table}.csv")


/tmp/ipython-input-2045735263.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables = pd.read_sql("SHOW TABLES;", conn)
/tmp/ipython-input-2045735263.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table};", conn)


Exported det_sales.csv
Exported ms_product.csv
Exported ms_sales.csv
Exported transaction.csv
